In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在 GCP 上的 E2E ML: MLOps 阶段 6：开始使用 Vertex AI 批量预测 AutoML 图像模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_image_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_automl_image_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_automl_image_model_batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程展示了如何使用Vertex AI SDK创建图像分类模型，并使用Google Cloud AutoML 模型进行批量预测。详情请点击[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)。

### 目标

在本教程中，您将学习如何从 Python 脚本中创建一个 AutoML 图像分类模型，然后使用 Vertex AI SDK 进行批量预测。您还可以选择使用 `gcloud` 命令行工具或在 Cloud 控制台上在线创建和部署模型。

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AutoML`
- `Vertex AI Batch Prediction`
- `Vertex AI Models`

所执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 训练一个 `AutoML` 图像分类模型。
- 使用 JSONL 输入进行批量预测。

在使用批量预测和在线预测时有一个关键区别：

* 预测服务：对整个数据集（即一个或多个数据项）执行即时预测并实时返回结果。

* 批量预测服务：对整个数据集进行排队（批量）预测，并在准备就绪时将结果存储在 Cloud Storage 存储桶中。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在本教程中使用的数据集版本存储在公共云存储桶中。训练模型可以预测图像中的花卉类型，包括五种花卉：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 费用

本教程使用谷歌云的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您的预测使用量生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex Workbench AI笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境符合本笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在Python 3虚拟环境中运行的Jupyter笔记本

[设置Python开发环境的云存储指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了详细的满足这些要求的说明。以下步骤提供了一组简化的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端窗口的命令行中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端窗口的命令行中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install tensorflow $USER_FLAG -q

### 重新启动内核

安装额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始前

### GPU 运行时

本教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐号时，您会获得$300 的免费信用额用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下方单元格中输入您的项目 ID。然后运行该单元格，确保Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意：** Jupyter 运行以`!`开头的行作为 shell 命令，并使用以`$`开头的 Python 变量进行插值。

设置您的项目ID

**如果您不知道您的项目ID**，您可能可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量用于笔记本的其余操作。以下为Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果你在一个在线教程会话中，你可能正在使用一个共享的测试账户或项目。为了避免在创建的资源之间发生命名冲突，你可以为每个实例会话创建一个uuid，并将其附加到你在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 对 Google Cloud 帐号进行认证

**如果您正在使用 Vertex AI Workbench 笔记本**，则您的环境已经通过认证。

**如果您正在使用Colab**，请运行下面的单元格，并按提示进行身份验证以通过oAuth认证您的帐号。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，然后选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，然后选择**存储对象管理员**。

点击创建。包含您的密钥的 JSON 文件将下载到本地环境中。

在下面的单元格中输入您服务帐号密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

当您为 Python 初始化 Vertex AI SDK 时，您需要指定一个云存储临时桶。这个临时桶是您的数据集和模型资源在不同会话中保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时才可以运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最终, 通过检查云存储桶的内容来验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

## 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 中的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在您已经准备好开始创建您自己的AutoML图像分类模型。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

本教程使用存储在公共云存储桶中的 Flowers 数据集的一个版本，使用 CSV 索引文件。

首先快速查看数据。您可以通过计算 CSV 索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：用于将数据项导入到`Dataset`资源中的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

该操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="Flowers" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建和运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练流水线，2）运行这个流水线。

#### 创建训练流水线

使用`AutoMLImageTrainingJob`类创建一个AutoML训练流水线，需要以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：要为其训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，则是单标签(`False`)还是多标签(`True`)。
- `model_type`：部署模型的类型。
  - `CLOUD`：部署在Google Cloud上
  - `CLOUD_HIGH_ACCURACY_1`：在Google Cloud上优化精度而非延迟的模型。
  - `CLOUD_LOW_LATENCY_`：在Google Cloud上优化延迟而非精度的模型。
  - `MOBILE_TF_VERSATILE_1`：部署在边缘设备上。
  - `MOBILE_TF_HIGH_ACCURACY_1`：在边缘设备上优化精度而非延迟的模型。
  - `MOBILE_TF_LOW_LATENCY_1`：在边缘设备上优化延迟而非精度的模型。
- `base_model`：（可选）从现有`Model`资源进行迁移学习，仅支持图像分类。

实例化的对象是训练任务的DAG（有向无环图）。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + UUID,
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

#### 执行训练流水线

接下来，通过调用方法`run`并使用以下参数来运行DAG以开始训练作业：

- `dataset`：要训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则在服务认为无法进一步改善模型目标测量之前，训练可能会在使用整个预算之前完成。

完成`run`方法后，将返回`Model`资源。

训练流水线的执行可能需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers_" + UUID,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

批量预测介绍

批量预测提供了离线批量处理大量预测请求的能力。资源只在批处理过程中进行预配，然后在批量请求完成时取消预配。结果存储在Cloud Storage中，与在线预测不同，后者将结果作为HTTP响应数据包返回。

批处理作业的输入格式取决于您的模型服务器支持的格式。首先，您的模型服务器中的网络服务器必须支持JSONL格式，该格式由网络服务器转换为模型输入接口或服务函数接口直接支持的格式。对于批量预测，这种JSONL格式被称为`pivot`格式。

批量预测作业的输入格式

批处理服务器接受以下输入格式的AutoML图像模型：

- JSONL

批处理服务器接受以下AutoML图像模型的输出格式：

- JSONL

Pivot格式

批处理服务器将输入格式转换为`pivot`（JSONL）格式如下：

**JSONL**

每个输入行（请求）应包含一个且仅有一个有效的json值。

    {"values": [1, 2, 3, 4], "key": 1}
    {"values": [5, 6, 7, 8], "key": 2}

批处理服务器使用相同格式生成pivot数据。生成的pivot数据然后包装到负载请求中：

    {"instances": [
      {"values": [1, 2, 3, 4], "key": 1},
      {"values": [5, 6, 7, 8], "key": 2}
    ]}

**CSV**

第一行中的csv头部总是会被忽略。字符串字段需要明确用双引号引起，否则该行将被丢弃，解析错误消息将被输出到错误文件。非引号值总是被传输为浮点数。

    col1,col2,col3
    1,3,"cat1"
    2,4,"cat2"

批处理服务器将每个输入行（请求）转换为JSON数组。

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**BigQuery**

每一行被转换为JSON数组。例如：

    [1.0,3.0,"cat1"]
    [2.0,4.0,"cat2"]

批处理服务器使用相同格式生成pivot数据。生成的pivot数据然后包装到负载请求中：

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**TFRecords**

TFRecord文件中的实例由apache_beam.io.tfrecordio模块以二进制形式读取。然后，二进制对象被序列化为ASCII字符串。预测服务器负责知道解码器以恢复实例。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

**FileList**

FileList格式包含文件列表。在“FileList”文件中，每一行指定一个文件路径，指定为Cloud Storage位置。

    gs://my-bucket/file1.txt
    gs://my-bucket/file2.txt

批处理服务器将文件读取为二进制文件。二进制对象被序列化为ASCII字符串。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

发送一个批量预测请求

将一个批量预测发送到您已部署的模型。

### 获取测试项目

现在对您的`AutoML`图像分类模型进行批量预测。您可以从数据集中随机选择一些示例作为测试项目。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2
if len(str(test_items[0]).split(",")) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(",")
    _, test_item_2, test_label_2 = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1 = str(test_items[0]).split(",")
    test_item_2, test_label_2 = str(test_items[1]).split(",")

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 复制测试项目

对于批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_URI/$file_1
! gsutil cp $test_item_2 $BUCKET_URI/$file_2

test_item_1 = BUCKET_URI + "/" + file_1
test_item_2 = BUCKET_URI + "/" + file_2

### 将批量输入文件制作为JSONL

现在制作一个批量输入文件，将其存储在您的本地云存储桶中。批量输入文件只能是JSONL格式。对于JSONL文件，您为每个数据项（实例）在每一行上创建一个字典条目。该字典包含键/值对：

- `content`：图像的云存储路径。
- `mime_type`：内容类型。在我们的示例中，是一个`jpeg`文件。

例如：

               {'content': '[your-bucket]/file1.jpg', 'mime_type': 'jpeg'}

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 发起批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `instances_format`：批量预测请求文件的格式："jsonl", "csv", "bigquery", "tf-record", "tf-record-gzip"或"file-list"。
- `prediction_format`：批量预测响应文件的格式："jsonl", "csv", "bigquery", "tf-record", "tf-record-gzip"或"file-list"。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，则调用会阻塞，直到异步批处理作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="flowers_" + UUID,
    instances_format="jsonl",
    predictions_format="jsonl",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

等待批量预测任务完成

接下来，等待批量任务完成。或者可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测任务完成。

In [ ]:
batch_predict_job.wait()

### 获取预测

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的 Cloud Storage 输出存储桶中。您可以调用方法 iter_outputs() 来获取包含结果的每个 Cloud Storage 文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `content`: 预测请求。
- `prediction`: 预测响应。
 - `ids`: 每个预测请求的内部分配的唯一标识符。
 - `displayNames`: 每个类别标签的类名。
 - `confidences`: 每个类别标签的预测置信度，介于0和1之间。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

您可以使用`delete()`方法删除您的`Vertex AI Batch Prediction`作业。

In [ ]:
batch_predict_job.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}